In [ ]:
%matplotlib inline

import math
import os
import sys
import datetime
import csv
import time
import copy
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
#import OrderedDicted to corectly align the network layers
#import nn you use activation and dropout features
from collections import OrderedDict
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import models
from torchvision import transforms
from torchvision import datasets
import torch.utils.data
import torch.utils.data as data
#from sampler import ImbalancedDatasetSampler
from torch.autograd import Variable
from torch.optim import lr_scheduler
from PIL import Image
import pandas as pd
import numpy as np
import tqdm
from IPython import display
from skimage import io, transform
import os
from torch.utils.data import Dataset, DataLoader
import matplotlib.image as mpimg
import scipy
from scipy import ndimage, misc
from scipy.misc import imshow
import skimage
from torch.utils.data.sampler import SubsetRandomSampler
import shutil
import pdb
import cv2

plt.ion()
use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")

In [ ]:
# x = torch.randn(2, 2, 3)
# print(x)
# print(x.size())
# x=x.view(3, 2,-1)
# print(x)
# print(x.size())

In [ ]:
trainlabels = pd.read_csv("/home/jupyter/kaggle_data/trainLabels.csv")

In [ ]:
class ImbalancedDatasetSampler(torch.utils.data.sampler.Sampler):
    def __init__(self, dataset, indices=None, num_samples=None):
                
        # if indices is not provided, 
        # all elements in the dataset will be considered
        self.indices = list(range(len(dataset))) \
            if indices is None else indices
            
        # if num_samples is not provided, 
        # draw `len(indices)` samples in each iteration
        self.num_samples = len(self.indices) \
            if num_samples is None else num_samples
            
        # distribution of classes in the dataset 
        label_to_count = {}
        for idx in self.indices:
            label = self._get_label(dataset, idx)
            if label in label_to_count:
                label_to_count[label] += 1
            else:
                label_to_count[label] = 1
                
        # weight for each sample
        weights = [1.0 / label_to_count[self._get_label(dataset, idx)]
                   for idx in self.indices]
        self.weights = torch.DoubleTensor(weights)

    def _get_label(self, dataset, idx):
        dataset_type = type(dataset)
        if dataset_type is torchvision.datasets.MNIST:
            return dataset.train_labels[idx].item()
        elif dataset_type is torchvision.datasets.ImageFolder:
            return dataset.imgs[idx][1]
        else:
            return dataset.imgs[idx][1]
                
    def __iter__(self):
        return (self.indices[i] for i in torch.multinomial(
            self.weights, self.num_samples, replacement=True))

    def __len__(self):
        return self.num_samples

In [ ]:
# this is copied from the works of pytorch (source: https://chsasank.github.io/vision/_modules/torchvision/datasets/folder.html)
IMG_EXTENSIONS = [
    '.jpg', '.JPG', '.jpeg', '.JPEG',
    '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP',
]

def is_image_file(filename):
    return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)

def find_classes(dir):
    classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
    classes.sort()
    class_to_idx = {classes[i]: i for i in range(len(classes))}
    return classes, class_to_idx

def make_dataset(dir, class_to_idx):
    images = []
    dir = os.path.expanduser(dir)
    for target in sorted(os.listdir(dir)): # all the directories of classes
        d = os.path.join(dir, target) # class directories are the targets
        if not os.path.isdir(d):
            continue

        for root, _, fnames in sorted(os.walk(d)): 
            for fname in sorted(fnames):
                if is_image_file(fname):
                    path = os.path.join(root, fname)
                    item = (path, class_to_idx[target])
                    images.append(item)

    return images


def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')


def accimage_loader(path):
    import accimage
    try:
        return accimage.Image(path)
    except IOError:
        # Potentially a decoding problem, fall back to PIL.Image
        return pil_loader(path)


def default_loader(path):
    from torchvision import get_image_backend
    if get_image_backend() == 'accimage':
        return accimage_loader(path)
    else:
        return pil_loader(path)


class DiabeticRetinoFolder(data.Dataset):
    """A generic data loader where the images are arranged in this way: ::

        root/dog/xxx.png
        root/dog/xxy.png
        root/dog/xxz.png

        root/cat/123.png
        root/cat/nsdf3.png
        root/cat/asd932_.png

    Args:
        root (string): Root directory path.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        loader (callable, optional): A function to load an image given its path.

     Attributes:
        classes (list): List of the class names.
        class_to_idx (dict): Dict with items (class_name, class_index).
        imgs (list): List of (image path, class_index) tuples
    """

    def __init__(self, root, transform=None, target_transform=None,
                 loader=default_loader):
        classes, class_to_idx = find_classes(root)
        imgs = make_dataset(root, class_to_idx)
        if len(imgs) == 0:
            raise(RuntimeError("Found 0 images in subfolders of: " + root + "\n"
                               "Supported image extensions are: " + ",".join(IMG_EXTENSIONS)))

        self.root = root
        self.imgs = imgs
        self.classes = classes
        self.class_to_idx = class_to_idx
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader

    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is class_index of the target class.
        """
        
        path, target = self.imgs[index]
        img = self.loader(path)
        if self.transform is not None:
            img = self.transform(img)
            #img = img.numpy()
        if self.target_transform is not None:
            target = self.target_transform(target)
            #target = target.numpy()

        return img, target


    def __len__(self):
        return len(self.imgs)

In [ ]:
data_dir = '/home/jupyter/kaggle_data/Train'
batch_size = 16

def dataset_DR(datadir, valid_size, batchsize):
    TRAIN_MEAN = np.array([0.485, 0.456, 0.406])
    TRAIN_STD = np.array([0.229, 0.224, 0.225]) 
    train_transform = transforms.Compose([
        transforms.CenterCrop(size=1024),
        transforms.Resize(1024),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize(TRAIN_MEAN, TRAIN_STD)])
    
    test_transform = transforms.Compose([
        transforms.Resize(size=1024),
        #transforms.CenterCrop(size=256),
        transforms.ToTensor(),
        transforms.Normalize(TRAIN_MEAN, TRAIN_STD)])
    
    train_data = DiabeticRetinoFolder(datadir, transform=train_transform)
    test_data = DiabeticRetinoFolder(datadir, transform=test_transform)
    
    trainloader = torch.utils.data.DataLoader(train_data, sampler=ImbalancedDatasetSampler(train_data), batch_size=batchsize, num_workers = 4)
    testloader = torch.utils.data.DataLoader(test_data, sampler=ImbalancedDatasetSampler(test_data), batch_size=batchsize, num_workers = 4)
    
    train_size = int((1-valid_size) * len(train_data))
    test_size = len(train_data) - train_size
    print("train and test sizes: ", train_size, ",", test_size)
    return train_data, test_data, trainloader, testloader, train_size, test_size

Trainset, Testset, TrainLoad, TestLoad, train_size, test_size = dataset_DR(data_dir, 0.1, batch_size)
classes = ('Mild_NPDR', 'Moderate_NPDR', 'Normal', 'PDR', 'Severe_NPDR')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
test_iter = iter(TestLoad)
%time image_batch, label_batch = next(test_iter) 
print(image_batch.shape, image_batch.type()) 
print(label_batch.shape, label_batch.type()) 
TRAIN_MEAN = np.array([0.485, 0.456, 0.406])
TRAIN_STD = np.array([0.229, 0.224, 0.225]) 
grid = torchvision.utils.make_grid(image_batch, nrow=8).cpu().numpy().transpose(1, 2, 0)
grid = TRAIN_STD * grid + TRAIN_MEAN
# The min and max values are very close to 0.0 and 1.0, but
# are just outside because of numerical effects.
grid = np.clip(grid, 0.0, 1.0)
plt.figure(figsize=(15, 15))
plt.imshow(grid)
#print(label_batch)

label_batch = label_batch.numpy().flatten()-1
print(label_batch)
print(' '.join('%4s' % classes[label_batch[j]] for j in range(batch_size)))

In [ ]:
v = models.resnet50(pretrained=True)
v.eval()

In [ ]:
class TrunkRes(torch.nn.Module):
  
  def __init__(self, orig_resnet):
    super().__init__()
    self.orig_resnet = orig_resnet
    child_counter = 0
    for child in orig_resnet.children():
        if child_counter < 9:
            #print("child ", child_counter, " was frozen")
            for param in child.parameters():
                param.requires_grad = False
        elif child_counter == 9:
            continue
        child_counter +=1
    
    self.conv1 = nn.Conv2d(3,64,7,stride=2,padding=3,bias=False)
    self.bn1 = nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.relu = nn.ReLU(inplace=True)
    self.maxpool = nn.MaxPool2d(3, stride=2, padding=1, dilation=1, ceil_mode=False)
    self.new_model = nn.Sequential(*list(orig_resnet.children())[:-1])
  
  def forward(self, x):
    #print("here: ",x.size())
    x = self.new_model(x)
    #print("there", x.size())
#     x = self.conv1(x)
#     x = self.bn1(x)
#     x = self.relu(x)
#     x = self.maxpool(x)
    
#     x = self.orig_resnet.layer1(x) #torch.Size([1, 256, 64, 64])
# #    print("layer1 output: ", x.size())
#     x = self.orig_resnet.layer2(x) #torch.Size([1, 512, 32, 32])
# #    print("layer2 output: ", x.size())
#     x = self.orig_resnet.layer3(x) # torch.Size([1, 1024, 16, 16])
# #    print("layer3 output: ", x.size())
#     x = self.orig_resnet.layer4(x) # torch.Size([1, 2048, 8, 8])
# #    print("layer4 output: ", x.size())
#     x = self.orig_resnet.avgpool(x) # torch.Size([1, 2048, 2, 2])
    #print("final output: ", x.size())
    return x

In [ ]:
# setting up my model with final layer customized
class HugeResnet(torch.nn.Module):
    def __init__(self, pretrained_model):
        super().__init__()
        self.pretrained_model = pretrained_model
        #num_final_in = pretrained_model.fc.in_features
        
        self.final_linear = torch.nn.Linear(131072, 5) # TODO
        self.logsoftmax = nn.LogSoftmax(dim=1)
    def forward(self, x):
        #print(x.size())
        x = x.view(x.size(0), -1)
        #print("new shape: ", x.size())
        x = self.final_linear(x)
        x = self.logsoftmax(x)
        #print("final shape: ", x.size())
        return x

In [ ]:
# # setting up my model with final layer customized
# class HugeResnet(torch.nn.Module):
#     def __init__(self, orig_resnet):
#         super().__init__()
#         self.orig_resnet = orig_resnet
#         #num_final_in = pretrained_model.fc.in_features
#         self.final_linear = torch.nn.Linear(1384448, 5) # TODO
#         self.logsoftmax = nn.LogSoftmax(dim=1)
#     def forward(self, x):
#         x = self.orig_resnet.conv1(x)
#         x = self.orig_resnet.bn1(x)
#         x = self.orig_resnet.relu(x)
#         x = self.orig_resnet.maxpool(x)

#         x = self.orig_resnet.layer1(x)
#         x = self.orig_resnet.layer2(x)
#         x = self.orig_resnet.layer3(x)
#         x = self.orig_resnet.layer4(x)

#         x = self.orig_resnet.avgpool(x)    
#         #print(x.size())
#         x = x.view(x.size(0), -1)
#         #print("new shape: ", x.size())
#         x = self.final_linear(x)
#         x = self.logsoftmax(x)
#         #print("final shape: ", x.size())
#         return x

In [ ]:
def resnet_on_sections(model, img):
    logits = []
    img = np.moveaxis(img, 0, -1) ##(1024, 1024, 3)
    for j in range(4):
        j = j+1
        for k in range(4):
            k = k+1
            img_sec = img[256*(k-1):256*k,256*(j-1):256*j,:]
            np_img = np.moveaxis(img_sec,2,0) #(3, 256, 256)
            np_img = torch.FloatTensor(np_img) # convert np array to tensor
            np_img= np_img.unsqueeze(0) # converts to a single batch torch.Size([1, 3, 256, 256])
            with torch.no_grad():
                Logit = model(np_img)
                Logit = np.array(Logit)
                Logit = np.reshape(Logit,(1,2,2,2048)) # (1, 2, 2, 2048)
                logits.append(Logit) 
    # once all 16 logits are collected per item in a logit list, so concat into a batchsizex31024x1024
    logits = np.array(logits) # (16, 1, 2, 2, 2048)
    return logits

In [ ]:
# x = torch.randn(3, 512, 512)
# print(x.size())

# x = x[0:3, 0:256, 0:256]
# print(x.size())

# x = x[0:3, 0:256, 0:512]
# print(x.size())

In [ ]:
# training function
def train_one_epoch(resn, big_resn):
    big_resn.eval()
    resn.eval()
    running_loss = 0.0
    running_corrects = 0
    count = 0
    Loss = []
    ER = [] #error rates
    for i, data in enumerate(TrainLoad, 0):
        count += 1
        image_batch, label_batch = data
        image_batch = Variable(image_batch.cuda())
        label_batch = Variable(label_batch.cuda())
        np_imgs = image_batch.cpu().numpy()
        label_batch = label_batch.cpu().numpy()
        label_batch = label_batch-1
        label_batch = Variable(torch.FloatTensor(label_batch).cuda())
        p_batch = label_batch.long()
        
        # zero the parameter gradients
        optimizer.zero_grad()
        Logitcollect = [] # batchsize units of logit terms
        for i in np_imgs: # per image in a batch
            Logits = resnet_on_sections(resn,i) #(16,1,2,2,2048)
            # once all 16 logits are collected per item in a logit list, so concat into a batchsizex31024x1024
            Logitcollect.append(Logits)

        input_batch = np.array(Logitcollect)
        # will have collected batch_size # of Logits in the Logitcollect list
        
        input_batch = Variable(torch.FloatTensor(input_batch).cuda())
#        print(input_batch.size()) # torch.Size([batchsize,16,1,2,2,2048])
        
        with torch.set_grad_enabled(True):
            logits = big_resn(input_batch).cuda()
            _, preds = torch.max(logits,1)
            #print(logits.size())
            #print(p_batch.size())
            #print(p_batch)
            
            loss = criterion(logits, p_batch).cuda()
            loss.backward()
            optimizer.step()
            
            #print statistics
            #print(preds)
            #print(preds.size())
            #print(label_batch.size())
            error_rate = 1.0 - (preds == label_batch.long()).float().mean()
            running_loss += loss.item() 
            running_corrects += torch.sum(preds == label_batch.long())
            #ER.append(error_rate)
            #print(i%1000 == 999)
#             if np.any(i % 1000 == 999):   # print every 1000 mini-batches
#                 print('[%5d] loss: %.3f' %(i+1, running_loss / 1000))
#                 running_loss = 0.0
    
#        mean_loss = float(sum(Loss))/float(count)
#        mean_error_rate=float(sum(ER))/float(count)
        print("training_error_rate: ", error_rate)
        return loss.item(), error_rate.item(), running_loss, running_corrects

In [ ]:
def val(resn, big_resn):
    correct = 0
    total = 0
    big_resn.eval()
    resn.eval()
    
    val_running_loss = 0.0
    val_running_corrects = 0
    count = 0
    ER = [] #error rates
    Loss = []
    for input_batch, label_batch in TestLoad:
        count += 1
        #input_batch = Variable(input_batch.cuda())
        np_imgs = input_batch.numpy()
        label_batch = Variable(label_batch.cuda())
        label_batch = label_batch.cpu().numpy()
        label_batch = label_batch-1
        label_batch = Variable(torch.FloatTensor(label_batch).cuda())
        p_batch = label_batch.long()
        
        Logitcollect = [] # batchsize units of logit terms

        for i in np_imgs: # for images in a batch
            Logits = resnet_on_sections(resn,i) #(16,1,2,2,2048)
            # once all 16 logits are collected per item in a logit list, so concat into a batchsizex31024x1024
            Logitcollect.append(Logits)
            
        outputs = np.array(Logitcollect)
        outputs = Variable(torch.FloatTensor(outputs).cuda())
        
        with torch.no_grad():
            logits = big_resn(outputs).cuda()
            _, preds = torch.max(logits,1)
            loss = criterion(logits, p_batch).cuda()
            
            #print statistics
            error_rate = 1.0 - (preds == label_batch.long()).float().mean()
            val_running_loss += loss.item()
            val_running_corrects += torch.sum(preds == label_batch.long())
            total += label_batch.size(0)
            correct += (preds == label_batch.long()).sum().item()
            
        print("Val_error_rate: ", error_rate)
        print('Accuracy of the network on the 841 test images: %d %%' % (100 * correct / total))
        return loss.item(), error_rate, val_running_loss, val_running_corrects,

In [ ]:
res50 = models.resnet50(pretrained=True)
MM = TrunkRes(res50)

tm = HugeResnet(v)
tm.to(device);

optimizer = optim.SGD(tm.parameters(), lr=0.001, momentum=0.9, nesterov=True)
weights = torch.tensor([0.24, 0.134, 0.03, 0.811, 1])
class_weights=torch.FloatTensor(weights).cuda()
criterion = nn.CrossEntropyLoss(weight=class_weights)

epoch = 0
epochs = []
train_losses, train_errors, train_run_losses, train_run_corrects = [], [], [], []
val_losses, val_errors, val_run_losses, val_run_corrects = [], [], [], []
num_epochs = 25

In [ ]:
best_acc = 0.0
best_model_wts = copy.deepcopy(tm.state_dict())
tm.load_state_dict(best_model_wts)
Best_model_Wts = copy.deepcopy(MM.state_dict())
MM.load_state_dict(Best_model_Wts)

Time = []

for epoch in range(num_epochs):
    since = time.time()
    
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 40)
    
    train_loss_epoch, mean_train_error_epoch, Train_runn_loss, Train_runn_corrects = train_one_epoch(MM, tm)
    val_loss_epoch, mean_val_error_epoch, Val_runn_loss, Val_runn_corrects = val(MM, tm)
    
    epoch += 1
    epochs.append(epoch)
    
    train_losses.append(train_loss_epoch)
    train_errors.append(mean_train_error_epoch)
    train_run_losses.append(Train_runn_loss)
    train_run_corrects.append(Train_runn_corrects)
#    train_epoch_loss = Train_runn_loss
    print(' Train # of Corrects : {:.4f} for a batchsize of : {:.4f}'.format(Train_runn_corrects, batch_size))
    train_epoch_acc = float(Train_runn_corrects) / float(batch_size)
    print(' Train Loss: {:.4f} Train Acc: {:.4f}'.format(train_loss_epoch, train_epoch_acc))
    
    val_losses.append(val_loss_epoch)
    val_errors.append(mean_val_error_epoch)
    val_run_losses.append(Val_runn_loss)
    val_run_corrects.append(Val_runn_corrects)
#    val_epoch_loss = Val_runn_loss
#    print(' Validation # of Corrects per batch : {:.4f}'.format(Val_runn_corrects))
    val_epoch_acc = float(Val_runn_corrects) / float(batch_size)
    print(' Val Loss: {:.4f} Val Acc: {:.4f}'.format(val_loss_epoch, val_epoch_acc))
    print(f'Epoch {epoch:3d}. Approx. train error rate: {mean_train_error_epoch:.3f}. Val error rate: {mean_val_error_epoch:.3f}.')
    
    if val_epoch_acc > best_acc:
        best_acc = val_epoch_acc
        best_model_wts = copy.deepcopy(tm.state_dict())
        tm.load_state_dict(best_model_wts)
    time_elapsed = time.time() - since
    print('Epoch complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    Time.append(time_elapsed)

fig, ax = plt.subplots(2, 1, sharex=True)
ax[0].plot(train_losses, label='train loss')
ax[0].plot(val_losses, label='val loss')
ax[0].legend()
ax[1].plot(train_errors, label='train error rate')
ax[1].plot(val_errors, label='val error rate')
ax[1].legend()
plt.grid(True)

Totaltime = sum(Time)
print('Training complete in {:.0f}m {:.0f}s'.format(Totaltime // 60, Totaltime % 60))
print('Best val Acc: {:4f}'.format(best_acc))

In [ ]:
plt.plot(train_run_corrects, label='# Correct Training prediction')
plt.plot(val_run_corrects, label='Validation prediction correct #')
plt.legend()
plt.xlabel("epochs")
plt.ylabel("# of corrects")
plt.grid(True)

In [ ]:
dataiter = iter(TestLoad)
images, labels = dataiter.next()
Labels = Variable(labels.cuda())
label_batch = Labels.float()
grid = torchvision.utils.make_grid(images, nrow=8).cpu().numpy().transpose(1, 2, 0)
grid = TRAIN_STD * grid + TRAIN_MEAN

# The min and max values are very close to 0.0 and 1.0, but
# are just outside because of numerical effects.
grid = np.clip(grid, 0.0, 1.0)
plt.figure(figsize=(15, 15))
plt.imshow(grid)
label_batch = label_batch.cpu().numpy()
label_batch = label_batch-1
label_batch = Variable(torch.FloatTensor(label_batch).cuda())
label_batch = label_batch.int()
print(label_batch)
print(' '.join('%5s' % classes[label_batch[j]] for j in range(batch_size)))

In [ ]:
class_correct = list(0. for i in range(5))
class_total = list(0. for i in range(5))

for input_batch, label_batch in TestLoad:
    np_imgs = input_batch.numpy()
    label_batch = Variable(label_batch.cuda())
    label_batch = label_batch.cpu().numpy()
    label_batch = label_batch-1
    label_batch = Variable(torch.FloatTensor(label_batch).cuda())
    p_batch = label_batch.long()

    Logitcollect = [] # batchsize units of logit terms

    for i in np_imgs: # for images in a batch
        Logits = resnet_on_sections(MM,i) #(16,1,2,2,2048)
        Logitcollect.append(Logits)

    outputs = np.array(Logitcollect)
    outputs = Variable(torch.FloatTensor(outputs).cuda())

    with torch.no_grad():
        logits = tm(outputs).cuda()
        _, preds = torch.max(logits,1)   
        c = (preds == label_batch.long()).squeeze()
        if len(label_batch) > 0:
            for i in range(len(c)):
                label = label_batch[i]
                print(c)
                class_correct[label.int().item()] += c[i].int().item()
                class_total[label.int().item()] += 1
        elif len(c) == 0:
            label = label_batch[0]
            class_correct[label.int().item()] += c.item()
            class_total[label.int().item()] += 1

        print(class_correct)
        print(class_total)

        for i in range(5):
            print('Accuracy of %5s : %2d %%' % (
                classes[i], 100 * class_correct[i] / class_total[i]))


In [ ]:
print('Predicted: ', ' '.join('%5s' % classes[preds[j]] for j in range(16)))
